In [1]:
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2023-01-20 08:34:01.313578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 08:34:01.417267: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-20 08:34:01.419807: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/subash/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-01-20 08:34:

In [2]:
data_dir = '../../data/PlantVillage/'

In [3]:
import pathlib
data_dir=pathlib.Path(data_dir)
data_dir

PosixPath('../../data/PlantVillage')

In [4]:
from os import listdir
img_dirs = listdir(data_dir)

In [5]:
for dirs in img_dirs:
    temp = list(data_dir.glob(dirs+'/*'))
    print(f"{dirs}: {len(temp)}")

Pepper__bell___Bacterial_spot: 997
Pepper__bell___healthy: 1478
Potato___Early_blight: 1000
Potato___healthy: 152
Potato___Late_blight: 1000
Tomato_Bacterial_spot: 2127
Tomato_Early_blight: 1000
Tomato_healthy: 1591
Tomato_Late_blight: 1909
Tomato_Leaf_Mold: 952
Tomato_Septoria_leaf_spot: 1771
Tomato_Spider_mites_Two_spotted_spider_mite: 1676
Tomato__Target_Spot: 1404
Tomato__Tomato_mosaic_virus: 373
Tomato__Tomato_YellowLeaf__Curl_Virus: 3209


In [6]:
dict = {}
label_dict = {}
i = 0
for dirs in img_dirs:
    dict[dirs] = list(data_dir.glob(dirs+'/*'))
    label_dict[dirs] = i
    i += 1

In [7]:
label_dict

{'Pepper__bell___Bacterial_spot': 0,
 'Pepper__bell___healthy': 1,
 'Potato___Early_blight': 2,
 'Potato___healthy': 3,
 'Potato___Late_blight': 4,
 'Tomato_Bacterial_spot': 5,
 'Tomato_Early_blight': 6,
 'Tomato_healthy': 7,
 'Tomato_Late_blight': 8,
 'Tomato_Leaf_Mold': 9,
 'Tomato_Septoria_leaf_spot': 10,
 'Tomato_Spider_mites_Two_spotted_spider_mite': 11,
 'Tomato__Target_Spot': 12,
 'Tomato__Tomato_mosaic_virus': 13,
 'Tomato__Tomato_YellowLeaf__Curl_Virus': 14}

In [8]:
X, y = [], []

for name, images in dict.items():
    i = 0
    for image in images:
        if i > 100:
            break
        i+=1
        img = cv2.imread(str(image))
        try:
            resized_img = cv2.resize(img,(255,255))
            X.append(resized_img)
            y.append(label_dict[name])
        except:
            print(f"debug: {image}")
            continue

In [9]:
X = np.array(X)
y = np.array(y)

In [10]:
len(X)

465

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [12]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [20]:
num_classes = 15
model = Sequential([
  layers.Conv2D(16, 2, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 2, padding='same', activation='relu'),
  #layers.MaxPooling2D(),
  #layers.Conv2D(64, 2, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  #layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=30)

Epoch 1/30
11/11 [==============================] - 3s 207ms/step - loss: 5.4295 - accuracy: 0.0891
Epoch 2/30
11/11 [==============================] - 2s 206ms/step - loss: 2.3947 - accuracy: 0.2701
Epoch 3/30
11/11 [==============================] - 2s 206ms/step - loss: 1.8354 - accuracy: 0.5632
Epoch 4/30
11/11 [==============================] - 2s 205ms/step - loss: 1.2282 - accuracy: 0.7644
Epoch 5/30
11/11 [==============================] - 2s 207ms/step - loss: 0.8211 - accuracy: 0.7874
Epoch 6/30
11/11 [==============================] - 2s 206ms/step - loss: 0.5008 - accuracy: 0.8937
Epoch 7/30
11/11 [==============================] - 2s 205ms/step - loss: 0.2867 - accuracy: 0.9454
Epoch 8/30
11/11 [==============================] - 2s 205ms/step - loss: 0.1903 - accuracy: 0.9799
Epoch 9/30
11/11 [==============================] - 2s 203ms/step - loss: 0.1022 - accuracy: 0.9943
Epoch 10/30
11/11 [==============================] - 2s 205ms/step - loss: 0.0561 - accuracy: 1.0000

In [21]:
model.evaluate(X_test_scaled,y_test)

4/4 [==============================] - 0s 56ms/step - loss: 1.2931 - accuracy: 0.6154


[1.2931426763534546, 0.6153846383094788]

In [22]:
predictions = model.predict(X_test_scaled)
predictions

4/4 [==============================] - 0s 55ms/step


array([[ -6.286304 ,  -2.6681347,  -4.127915 , ...,  -5.0609455,
         -0.7557312,  -5.095532 ],
       [ -6.5605307,  -8.750807 , -15.737734 , ...,   0.8144531,
        -10.587662 ,  -5.3414183],
       [-11.991996 , -14.501716 , -15.216777 , ...,  -8.14726  ,
        -16.306957 ,  -9.665678 ],
       ...,
       [ -7.251989 , -12.422585 , -15.819239 , ..., -10.059768 ,
        -13.124697 ,  -1.4410814],
       [ -5.006123 ,  -6.3970942,  -8.694085 , ..., -11.913518 ,
         -5.457448 , -13.1413555],
       [-16.860046 , -32.57868  , -13.873348 , ..., -14.994182 ,
        -23.268589 , -29.56153  ]], dtype=float32)

In [23]:
import pickle

In [25]:
pickle.dump(model, open("../../models/dide1.pkl", 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......flatten
.........vars
......max_pooling2d
.........vars
......max_pooling2d_1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
variables.h5                                   2023-01-20 01:48:38     22919328
config.json                                    2023-01-20 01:48:38         2863
metadata.json                                  2023-01-20 01:48:38           64


In [27]:
loaded_model = pickle.load(open("../../models/dide1.pkl", 'rb'))

Keras model archive loading:
File Name                                             Modified             Size
variables.h5                                   2023-01-20 01:48:38     22919328
config.json                                    2023-01-20 01:48:38         2863
metadata.json                                  2023-01-20 01:48:38           64
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......flatten
.........vars
......max_pooling2d
.........vars
......max_pooling2d_1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


In [28]:
loaded_model.evaluate(X_test_scaled,y_test)

4/4 [==============================] - 0s 55ms/step - loss: 1.2931 - accuracy: 0.6154


[1.2931426763534546, 0.6153846383094788]